In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn import tree


In [2]:
df_full = pd.read_csv('../data/compas-scores-two-years.csv')
display(df_full)  #kaller df full fordi vi vil endre på et stort datasett, men ikke intressert i 
#å bruke hele, slik at vi lager nytt og da må vi kunne referer til både gammelt nytt. 
#Kaller dette datasettet da df_full, og neste bare df.


,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7209,10996,steven butler,steven,butler,2013-11-23,Male,1992-07-17,23,Less than 25,African-American,...,5,Medium,2013-11-23,2013-11-22,2013-11-24,0,1,860,0,0
7210,10997,malcolm simmons,malcolm,simmons,2014-02-01,Male,1993-03-25,23,Less than 25,African-American,...,5,Medium,2014-02-01,2014-01-31,2014-02-02,0,1,790,0,0
7211,10999,winston gregory,winston,gregory,2014-01-14,Male,1958-10-01,57,Greater than 45,Other,...,1,Low,2014-01-14,2014-01-13,2014-01-14,0,0,808,0,0
7212,11000,farrah jean,farrah,jean,2014-03-09,Female,1982-11-17,33,25 - 45,African-American,...,2,Low,2014-03-09,2014-03-08,2014-03-09,3,0,754,0,0


In [3]:
df_full.columns # viser ALLE kolonner

Index(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob',
       'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number',
       'c_offense_date', 'c_arrest_date', 'c_days_from_compas',
       'c_charge_degree', 'c_charge_desc', 'is_recid', 'r_case_number',
       'r_charge_degree', 'r_days_from_arrest', 'r_offense_date',
       'r_charge_desc', 'r_jail_in', 'r_jail_out', 'violent_recid',
       'is_violent_recid', 'vr_case_number', 'vr_charge_degree',
       'vr_offense_date', 'vr_charge_desc', 'type_of_assessment',
       'decile_score.1', 'score_text', 'screening_date',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text',
       'v_screening_date', 'in_custody', 'out_custody', 'priors_count.1',
       'start', 'end', 'event', 'two_year_recid'],
      dtype='object')

In [4]:
#trenger ikke bruke alle kolonner. Lager derfor ny df. Lager det ved å skrive pd.dataframe
#dictionary som viser at dataene til kolonne sex ligger i kolonne sex, ved å referere til 
# gammel dataframe df_full. Får under her tilgang på hvit-ikkehvit i binære verdiene. 
# scorene fra 1-10 vet vi fra før at deles inn i low, medium og high. Fra oppgaveteksten 
# vet vi at medium og high er en felles kategori. Vi har med det bare to kategorier, og pedicted blir
# dermed true falce likevel.

df = pd.DataFrame({         
    'sex': df_full.sex, #kaller kolonne"
    'white': df_full.race == 'Caucasian', #har i tillegg definert at enten er den lik eller ikke lik hvit
    'predicated_score': df_full.decile_score,  #kaller kolonnen som allerede er der 
    'predicted': df_full.score_text != 'Low', #hvis den ikke er low, da er den sann, og det 
    'actual': df_full.is_recid == 1,          #ˆbetyr at de er predikert til å begå lovbrudd
})  #ˆom de begår lovbrudd eller ikke på actual. Vil også ha true og false her, 1 er sann 

df



,sex,white,predicated_score,predicted,actual
0,Male,False,1,False,False
1,Male,False,3,False,True
2,Male,False,4,False,True
3,Male,False,8,True,False
4,Male,False,1,False,False
...,...,...,...,...,...
7209,Male,False,7,True,False
7210,Male,False,3,False,False
7211,Male,False,1,False,False
7212,Female,False,2,False,False


In [6]:
#herunder henter vi ut alle som er hvite og teller dem. Vi henter også ut antallet som ble 
#predicted til å begå lovbrudd OG er hvite. Og så deler vi, for å finne prosenten av hvor mange
#hvite som vil begå lovbrudd gitt at de er hvite, det andre antallet på det første

def get_relative_frequency(group, variable):
    group_size = len(df[group]) #bruker dataene fra kolonnen som er inni klammene til 
    #å filtrere radene, fordi vi vil bare ha de radene hvor group er true. Med len får 
    #vi på en måte størrelsen
    filtered_group_size = len(df[variable & group]) 
    return filtered_group_size/group_size

get_relative_frequency(df.white, df.predicted) 

0.3480032599837001

In [ ]:
#herunder gjør vi det samme ovenfor bare for ikkehvite

In [7]:
#for hvite predicted og hvite actual ser vi at det var flere som ble predicted enn det var
#actual, og på ikke-hvite så er tallet ganske likt. Dette er det vi vet hittil. Nå vil vi 
#lage en fin tabell. Da må vi ha ny dataframe. Vi vil ha hvit og ikke-hvit som rader,
#og predicted og actual som kolonner. Lager da dataframe med dict. 

pd.Dataframe({
    'Predicted':[], #dette er en liste med navn på radene. listen har to verdier. 
    #det er ulike måter å sende data inn, men vi bruker dict hvor nøkkelen er navnet på 
    #kollonnen og verdien er verdien til kollonnen. 
    "Actual": [],
    
#Under gir vi navn på radene våre    
}, index = ['White', 'Non-White']) 

In [ ]:
#herunder gjør vi det samme for å finne i forhold til kjønn. lager ny dataframe. Tallene vi
#får indikerer at kvinner er overpredikert og menn er underpredikert i at de skal begå
#nye lovbrudd

In [ ]:
#NESTE PUNKT 

In [8]:
categorical_cols = ['sex', 'race', 'v_score_text']

qf = pd.get_dummies(df, columns=categorical_cols, prefix=categorical_cols, prefix_sep='_')

statsOne = qf[['sex_Male','race_African-American', 'race_Caucasian']]
score = qf['score_text']

KeyError: "['v_score_text', 'race'] not in index"

In [ ]:
qf.plot.bar(x=statsOne, y=score)